In [2]:
config_file = 'delineation/configs/dislocation_matching_disp_and_warp_and var_home.yml'
aug_config_file = 'delineation/configs/aug.yml'
import torch
from delineation.configs.defaults_segmentation import _C as cfg
from delineation.datasets import make_data_loader
from delineation.models import build_model_list
from delineation.utils import settings, cost_volume_helpers
from delineation.utils.settings import evaluate_results

from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

import yaml

cfg.merge_from_file(config_file)
with open(aug_config_file, 'r') as ymlfile:
    cfg_aug = yaml.load(aug_config_file)

_device = settings.initialize_cuda_and_logging(cfg)  # '_device' is GLOBAL VAR

train_loader, val_loader = make_data_loader(cfg, cfg_aug)
seg_model, model = build_model_list(cfg, True)

/home/dagnyt/EPFL/dislocations/delineation/layers/loss.py:6: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('agg')
/home/dagnyt/EPFL/dislocations/delineation/utils/settings.py:8: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('agg')


Using cuda
unet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/dislocations_segmentation/150_segmentor.tar' (epoch 150)
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_04/dislocations_matching_disp_warp_var_joint_10addaug/150_seg.tar' (epoch 150)
scmnet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_04/dislocations_matching_disp_warp_var_joint_10addaug/150_scmnet_light.tar' (epoch 150)


In [4]:
seg_model.eval()
model.eval()
total_test_loss = 0
pix1_err_m, pix3_err_m, pix5_err_m, epe_m, count = 0, 0, 0, 0, 0
from delineation.utils import settings, cost_volume_helpers

for batch_idx, (l, r, lgt, rgt, dlgt, l_name) in (enumerate(val_loader)):
        indices = cost_volume_helpers.volume_indices(2 * cfg.TRAINING.MAXDISP, len(l),
                                                     cfg.TRAINING.HEIGHT, cfg.TRAINING.WIDTH, _device)

        with torch.no_grad():
            l, r, lgt, rgt, dlgt = l.to(_device), r.to(_device), lgt.to(_device), rgt.to(_device), dlgt.to(_device)

            l_seg, l_segmap = seg_model(l)
            r_seg, r_segmap = seg_model(r)
            if abs(dlgt.max()>32) or abs(dlgt.min()>32):
                continue
                
            print(l_name)
            dl_scores = model(l_segmap, r_segmap)
            
            dl_ = F.softmax(-dl_scores, 2)
            dl = torch.sum(dl_.mul(indices), 2) - cfg.TRAINING.MAXDISP
            for i in range(0, len(dlgt)):
                pix1_err, pix3_err, pix5_err, epe = evaluate_results(dlgt[i], dl[i], lgt[i])
                print(pix1_err, pix3_err, pix5_err, epe, dlgt.min(), dlgt.max())
                pix1_err_m += pix1_err
                pix3_err_m += pix3_err
                pix5_err_m += pix5_err
                epe_m += epe
                count += 1
            torch.cuda.empty_cache()
            
            lgt = lgt.cpu().numpy()
            dl = dl.detach().permute(0, 2, 3, 1).cpu().numpy()
            dlgt = dlgt.unsqueeze(1).detach().permute(0, 2, 3, 1).cpu().numpy()

            for i in tqdm(range(len(dl))):
                vol = cost_volume_helpers.back_project_numpy(lgt[i,0,:,:], dl[i,:,:,:], cfg.TRAINING.MAXDISP, mode='two-sided')
                vol_gt = cost_volume_helpers.back_project_numpy(lgt[i,0,:,:], dlgt[i,:,:,:], cfg.TRAINING.MAXDISP, mode='two-sided')

                for idx, rotation_angle in enumerate(range(-20, 20, 1)):
                    cost_volume_helpers.visualize_volume(l_name[i], vol[0, :, :, :],
                                                         rotation_angle,
                                                         cfg.LOGGING.LOG_DIR,
                                                         mode='scatter',
                                                         save_ext=idx,
                                                         plot=False)

                    cost_volume_helpers.visualize_volume(l_name[i], vol_gt[0, :, :, :],
                                                         rotation_angle,
                                                         cfg.LOGGING.LOG_DIR+'_gt',
                                                         mode='scatter',
                                                         save_ext=idx,
                                                         plot=False)

                torch.cuda.empty_cache()

            
print('here')
values = pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count
print('Mean per dataset: {}, {}, {}, {}'.format(pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count))

('1116_1116 DPC DF4 71.0 kx DF4-B_LEFT.png',)


  0%|          | 0/1 [00:00<?, ?it/s]

0.16789667896678967 0.003186850050318685 0.0005031868500503186 0.52003753 tensor(-4., device='cuda:0') tensor(2., device='cuda:0')
[[-1.1124249  -0.9420433  -1.0258274  ... -1.8389931  -1.5169563
  -2.2741013 ]
 [-1.7532749  -1.4341583  -1.5340557  ...  0.9021225  -1.5668869
   0.00769043]
 [-1.3308334  -1.2200966  -1.4194889  ... -0.54496956 -1.3075829
  -1.9120541 ]
 ...
 [-1.0667877  -1.8552666  -1.4779587  ... -1.1177692   0.21821594
  -1.9547386 ]
 [-1.258812   -1.8837719  -1.4870453  ... -0.88077545 -1.1844444
  -0.09703636]
 [-1.22398    -0.52303314 -1.8397274  ...  1.137043    0.5808945
   0.90034866]]
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
[[-1.1124249  -0.9420433  -1.0258274  ... -1.8389931  -1.5169563
  -2.2741013 ]
 [-1.7532749  -1.4341583  -1.5340557  ...  0.9021225  -1.5668869
   0.00769043]
 [-1.3308334  -1.2200966  -1.4194889  ... -0.54496956 -1.3075829
  -1.9120541 ]
 ...
 [-1.0667877  -1.

100%|██████████| 1/1 [00:36<00:00, 36.97s/it]


('1123_1123 DPC DF4 71.0 kx DF4-B_LEFT.png',)


...

('pairs_18_20_GaN - TiltAxis is almost vertical_0018_LEFT.png',)


  0%|          | 0/1 [00:00<?, ?it/s]

0.07107742728390004 0.004916018025399427 0.00040966816878328555 0.49684608 tensor(-1., device='cuda:0') tensor(4., device='cuda:0')
[[ 0.76488113  0.36078262  0.07135773 ... -0.7664108  -0.60871506
  -1.698431  ]
 [ 0.5832596   0.63752747 -0.01145935 ...  4.427265    0.25294876
   0.42229462]
 [ 0.7178383   0.4440956   0.21625519 ...  1.4960823  -1.328949
  -0.70261955]
 ...
 [-1.5591164  -2.2200794  -1.97686    ...  2.3498688   4.2507706
   0.16420746]
 [-1.8200912  -2.214323   -1.947525   ...  1.5847816   0.94887924
   0.7436447 ]
 [-1.6532173  -1.7428322  -2.3545628  ...  2.4215164   3.0461197
   0.43989563]]
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
[[ 0.76488113  0.36078262  0.07135773 ... -0.7664108  -0.60871506
  -1.698431  ]
 [ 0.5832596   0.63752747 -0.01145935 ...  4.427265    0.25294876
   0.42229462]
 [ 0.7178383   0.4440956   0.21625519 ...  1.4960823  -1.328949
  -0.70261955]
 ...
 [-1.5591164  

100%|██████████| 1/1 [00:49<00:00, 49.91s/it]

here
Mean per dataset: 0.424544916019207, 0.12442332210611107, 0.07683120224707464, 2.1063171699643135
